In [122]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

# external files
import augmentations
import bilinear_sampler
import dataset
from dataset import Dataset
import dataloader
from dataloader import DataLoader
import losses
import model
import utils

import importlib
importlib.reload(augmentations)
importlib.reload(bilinear_sampler)
importlib.reload(dataset)
importlib.reload(dataloader)
importlib.reload(losses)
importlib.reload(model)
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\chm1clj\\Desktop\\work\\monodepth-estimation\\utils.py'>

In [3]:
DATA_DIR_LEFT='C:/Users/chm1clj/Desktop/work/dataset/leftImg8bit/'
DATA_DIR_RIGHT='C:/Users/chm1clj/Desktop/work/dataset/rightImg8bit/'
DATA_DIR_DISP='C:/Users/chm1clj/Desktop/work/dataset/disparity/'

l_train_dir=os.path.join(DATA_DIR_LEFT,'train')
l_val_dir=os.path.join(DATA_DIR_LEFT,'val')
l_test_dir=os.path.join(DATA_DIR_LEFT,'test')

r_train_dir=os.path.join(DATA_DIR_RIGHT,'train')
r_val_dir=os.path.join(DATA_DIR_RIGHT,'val')
r_test_dir=os.path.join(DATA_DIR_RIGHT,'test')

disp_train_dir=os.path.join(DATA_DIR_DISP,'train')
disp_val_dir=os.path.join(DATA_DIR_DISP,'val')
disp_test_dir=os.path.join(DATA_DIR_DISP,'test')

In [80]:
"""Load dataset"""
# Train Dataset & Dataloader
train_dataset=Dataset(
    mode='train',
    left_img_dir=l_train_dir,
    right_img_dir=r_train_dir,
    augmentation=True,
    )
train_dataloader=DataLoader(
    mode='train',
    dataset=train_dataset,
    batch_size=8,
    shuffle=True
)

# Val Dataset & Dataloader
val_dataset=Dataset(
    mode='train',
    left_img_dir=l_val_dir,
    right_img_dir=r_val_dir,
    augmentation=True,
    )
val_dataloader=DataLoader(
    mode='train',
    dataset=val_dataset,
    batch_size=8,
    shuffle=True
)

# Test Dataset & Dataloader
test_dataset=Dataset(
    mode='test',
    left_img_dir=l_test_dir,
    augmentation=True,
)
test_dataloader=DataLoader(
    mode='test',
    dataset=test_dataset,
    batch_size=8,
    shuffle=True
)

In [81]:
x_true,y_true=train_dataloader[0]
print(x_true.shape,y_true.shape)

(8, 1024, 2048, 3) (8, 1024, 2048, 6)


In [123]:
def MSE(y_true, y_pred):
    l_true=y_true[0]
    r_true=y_true[1]

    l_disp_pred=y_pred[...,0]
    r_disp_pred=y_pred[...,1]

    l_disp_pred_neg=tf.math.negative(l_disp_pred)
    l_recons=bilinear_sampler.billinear_sampler_fct(r_true,l_disp_pred_neg)
    r_recons=bilinear_sampler.billinear_sampler_fct(l_true,r_disp_pred)

    loss_l = mean(square(l_true - l_recons), axis=-1)
    loss_r = mean(square(r_true - r_recons), axis=-1)
    loss=(loss_l+loss_r)/2
    return loss

In [22]:
    unet_model=model.UNet(
        input_size=(1024,2048,3),
        num_outputs=2,
        num_levels=4,
        residual_blocks_size=[3,3,3,3],
        filters_size_start=64,
        batch_norm=True,)
    # unet_model.summary()

In [132]:
@tf.function
def total_loss_function(y_true, y_pred):
    alpha_image_loss=1.0/3
    disp_gradient_loss_weight=1.0/3
    lr_loss_weight=1.0/3

    l_true=y_true[:,:,:,:3]
    r_true=y_true[:,:,:,3:]

    print('input img shape',y_true.shape)
    print('left shape',l_true.shape)
    print('pred shape',y_pred.shape)

    l_disp_pred=y_pred[...,0]
    l_disp_pred=tf.expand_dims(l_disp_pred, axis=3)
    r_disp_pred=y_pred[...,1]
    r_disp_pred=tf.expand_dims(r_disp_pred, axis=3)

    # l_disp_pred_neg=[-x for x in l_disp_pred]
    l_disp_pred_neg=tf.math.negative(l_disp_pred)
    l_recons=bilinear_sampler.billinear_sampler_fct(r_true,l_disp_pred_neg)
    r_recons=bilinear_sampler.billinear_sampler_fct(l_true,r_disp_pred)

    print("reconstructed shape",l_recons.shape)

    loss=losses.total_loss(
        alpha_image_loss,
        disp_gradient_loss_weight,
        lr_loss_weight,
        l_true,
        l_recons,
        l_disp_pred,
        r_true,
        r_recons,
        r_disp_pred
    )

    return loss

In [133]:
learning_rate=0.0001
optimizer=keras.optimizers.Adam(learning_rate)
metrics=[MSE]

unet_model.compile(
    optimizer=optimizer,
    loss=total_loss_function,
    # metrics=metrics
)

input img shape (None, None, None, None)
left shape (None, None, None, None)
pred shape (None, 1024, 2048, 2)
reconstructed shape (None, 1024, 2048, 3)


TypeError: in converted code:

    <ipython-input-132-b66964d4ac57>:26 total_loss_function  *
        loss=losses.total_loss(
    c:\Users\chm1clj\Desktop\work\monodepth-estimation\losses.py:154 total_loss  *
        Cds=disparity_smoothness_loss(l_true,l_disp_pred,r_true,r_disp_pred)
    c:\Users\chm1clj\Desktop\work\monodepth-estimation\losses.py:108 disparity_smoothness_loss  *
        loss=disparity_smoothness_loss_single(l_true[i], l_disp_pred[i],r_true[i],r_disp_pred[i])
    c:\Users\chm1clj\Desktop\work\monodepth-estimation\losses.py:95 disparity_smoothness_loss_single  *
        l_disp=get_disparity_smoothness(l_true,l_disp_pred)
    c:\Users\chm1clj\Desktop\work\monodepth-estimation\losses.py:77 get_disparity_smoothness  *
        img_gradient_x=gradient_x(img)
    c:\Users\chm1clj\Desktop\work\monodepth-estimation\losses.py:63 gradient_x  *
        zero_col=tf.zeros((1024,1,img.shape[2]),dtype=tf.dtypes.float32)
    C:\Users\chm1clj\.conda\envs\myenvironment\lib\site-packages